<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/Integrating_Multiple_Agentic_Frameworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Integrating Multiple Agentic Frameworks

In this tutorial, we explore how to combine LangGraph, a flexible framework for building agentic workflows, with AutoGen, a powerful agent system from Microsoft, to build an intelligent chatbot that:

Supports persistent memory between interactions

Uses AutoGen's reasoning abilities to handle complex tasks

Leverages LangGraph's features like task routing, entrypoints, and streaming output

##Why integrate LangGraph with other frameworks?

LangGraph provides strong abstractions for stateful, multi-step applications. Integrating external agent frameworks like AutoGen enables:

Modular agent design across ecosystems
Seamless use of LangGraph features (memory, streaming, persistence)
Support for hybrid multi-agent workflows


##Step 1: Install Dependencies

In [1]:
%pip install autogen langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 834.0/834.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.7 MB/s eta 0:00:00


##Step 2: Configure OpenAI Key

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY: ··········


##Step 3: Define AutoGen Agents

We’ll use AutoGen's AssistantAgent and UserProxyAgent. These agents can auto-respond and run code when needed.

In [3]:
import autogen
import os

config_list = [{"model": "gpt-4o", "api_key": os.environ["OPENAI_API_KEY"]}]

llm_config = {
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
    "temperature": 0,
}

autogen_agent = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "web",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    llm_config=llm_config,
    system_message="Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.",
)

##Step 4: Create the LangGraph Workflow

Let’s build a LangGraph graph that delegates reasoning to AutoGen, with support for short-term memory and checkpointing.


In [4]:
from langchain_core.messages import convert_to_openai_messages, BaseMessage
from langgraph.func import entrypoint, task
from langgraph.graph import add_messages
from langgraph.checkpoint.memory import MemorySaver


@task
def call_autogen_agent(messages: list[BaseMessage]):
    # convert to openai-style messages
    messages = convert_to_openai_messages(messages)
    response = user_proxy.initiate_chat(
        autogen_agent,
        message=messages[-1],
        # pass previous message history as context
        carryover=messages[:-1],
    )
    # get the final response from the agent
    content = response.chat_history[-1]["content"]
    return {"role": "assistant", "content": content}


# add short-term memory for storing conversation history
checkpointer = MemorySaver()


@entrypoint(checkpointer=checkpointer)
def workflow(messages: list[BaseMessage], previous: list[BaseMessage]):
    messages = add_messages(previous or [], messages)
    response = call_autogen_agent(messages).result()
    return entrypoint.final(value=response, save=add_messages(messages, response))

###  Step 5: Run the Graph

Initialize a Config with Thread ID

In [5]:
# pass the thread ID to persist agent outputs for future interactions
# highlight-next-line
config = {"configurable": {"thread_id": "2"}}

for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Which numbers between 1 and 50 are divisible by 7?",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)

user_proxy (to assistant):

Which numbers between 1 and 50 are divisible by 7?

--------------------------------------------------------------------------------
assistant (to user_proxy):

To find the numbers between 1 and 50 that are divisible by 7, we can iterate through the numbers in this range and check if each number is divisible by 7. Here's a simple Python script to do that:

```python
# filename: divisible_by_7.py

def find_divisible_by_7():
    divisible_numbers = [number for number in range(1, 51) if number % 7 == 0]
    print("Numbers between 1 and 50 that are divisible by 7:", divisible_numbers)

find_divisible_by_7()
```

This script will output the numbers between 1 and 50 that are divisible by 7. Please execute the code to see the result.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 
Numbers bet

### **Step 6: Continue the Conversation**
Because we're using LangGraph’s persistence layer, we can pick up the thread in future runs:


In [6]:
for chunk in workflow.stream(
    [
        {
            "role": "user",
            "content": "Multiply the last number by 3",
        }
    ],
    # highlight-next-line
    config,
):
    print(chunk)

user_proxy (to assistant):

Multiply the last number by 3
Context: 
Which numbers between 1 and 50 are divisible by 7?
The numbers between 1 and 50 that are divisible by 7 are: 7, 14, 21, 28, 35, 42, and 49. 

TERMINATE

--------------------------------------------------------------------------------
assistant (to user_proxy):

The last number in the list of numbers between 1 and 50 that are divisible by 7 is 49. To multiply this number by 3:

49 * 3 = 147

Therefore, the result is 147.

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (39dab331-cbda-447a-9e2a-f50ec0a57c36): Termination message condition on agent 'user_proxy' met
{'call_autogen_agent': {'role': 'assistant', 'content': 'The last number in the list of numbers between 1 and 50 that are divisible by 7 is 49. To multiply this number by 3:\n\n49 * 3 = 147\n\nTherefore, the result is 147.\n\nTERMINATE'}}
{'workflow': {'role': 'assistant', 'content': 'The las